In [24]:
from math import log
from pandas import DataFrame,Series
import pandas as pd
import operator
import numpy as np
import matplotlib.pyplot as plt

dataSet = pd.read_csv('C:/Users/YT/Desktop/train(1).csv')
#print(dataSet.columns)
#print(dataSet.head(5))
dataSet=dataSet.drop(['PassengerId'],axis=1)
#print(dataSet.head(5))

s = dataSet['Fare']
for i in range(len(dataSet)):
    if s[i]<7.910400:
        dataSet.ix[i, 'Fare2']=0
    elif s[i]<14.454200:
        dataSet.ix[i, 'Fare2']=1
    elif s[i]<31.000000:
        dataSet.ix[i, 'Fare2']=2
    else:
        dataSet.ix[i, 'Fare2']=3
    if dataSet.ix[i, 'Parch']>2:
        dataSet.ix[i, 'Parch'] = 2
    if dataSet.ix[i, 'SibSp']>3:
        dataSet.ix[i, 'SibSp'] = 3


#print(dataSet['Age'])
k=dataSet['Age']
for i in range(len(dataSet)):
    if k[i]=='NaN':
        dataSet=dataSet.dropna(axis=0,subset=['Age'])
    elif k[i]<5.000000:
        dataSet.ix[i,'Age2']=0
    elif s[i]<15.000000:
        dataSet.ix[i,'Age2']=1
    elif s[i]<60.000000:
        dataSet.ix[i,'Age2']=2
    else:
        dataSet.ix[i,'Age2']=3
        
#print(dataSet['Age2'])

#删除无用变量
dataSet=dataSet.drop(['Age'],axis=1)
dataSet=dataSet.drop(['Cabin'],axis=1)
dataSet=dataSet.drop(['Name'],axis=1)
dataSet=dataSet.drop(['Ticket'],axis=1)
dataSet=dataSet.drop(['Fare'],axis=1)
print(dataSet.head(3))

dataSet=dataSet.fillna({'Embarked':'S'})
#dataSet=np.array(dataSet)
#print(dataSet)


C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc fo

   Survived  Pclass     Sex  SibSp  Parch Embarked  Fare2  Age2
0         0       3    male      1      0        S    0.0   1.0
1         1       1  female      1      0        C    3.0   3.0
2         1       3  female      0      0        S    1.0   1.0


In [25]:
#计算数据集的熵值确定最佳判断的属性

def sulEnt(data):
    numEnt=len(data)
    labelsCount={}
    l=data.columns.tolist()
   #print(l)
    #print(data.head(3))
    k=l.index('Survived')
    #print(k)
    for i in range(len(data)):
        featVec=data.loc[i]
        #print(featVec)
        #featVec=list(featVec)
        currentLabel=featVec[k]
        #print(currentLabel)
        if currentLabel not in labelsCount.keys():
            labelsCount[currentLabel]=0
        labelsCount[currentLabel]+=1
    resultEnt=0.0
    for key in labelsCount:
        p=float(labelsCount[key])/numEnt
        resultEnt-=p*log(p,2)
    return resultEnt

#mydata=sulEnt(dataSet)
#print(mydata)

In [26]:
#按照给定的属性划分数据集,将同一属性的各种不同取值进行分类，以便于计算增益熵中的权重
def splitData(data,axis,value):
    retData=data[data[axis]==value]
    retData=retData.drop([axis],axis=1)
    return retData
#mydata=splitData(dataSet,'Pclass',1)
#print(mydata)

In [29]:
#计算不同属性的增益熵，决定最有分类属性进行分类
from math import log
def chooseBestFeatureToSplit(data):
    baseEnt=sulEnt(data)
    bestEnt=0.0
    bestFeature=data.columns[1]
    for i in data.columns:
        if i=='Survived':continue
        featList=data[i]
        #print(featList)
        uniqueVals=set(featList)
        #print(uniqueVals)
        newEnt=0.0
        for value in uniqueVals:
            featSplit=splitData(data,i,value)
            #print(featSplit)
            r=len(data[data[i]==value])/float(len(data))
           # print(r)
            featSplit=featSplit.reset_index()
            featSplit=featSplit.drop(['index'],axis=1)
            #print(featSplit)
            newEnt+=r*sulEnt(featSplit)
            #print(newEnt)
        infoGain=baseEnt-newEnt
        #if i !='Survived':
        if (infoGain>bestEnt):
            bestEnt=infoGain
            bestFeature=i
    return bestFeature

#mydata=chooseBestFeatureToSplit(dataSet)
#print(mydata)

In [30]:
#判断叶子结点中所有样本是否属于同一类，如不属于同一类则统计其中类别个数最多的哪一类作为该叶子节点的分类，并返回
import operator
def majorityClass(classList):
    classCount={}
    for i in classList:
        if i not in classCount.keys():
            classCount[i]=0
        classCount[i]+=1
    sortClass=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortClass[0][0]

In [31]:
#利用递归方法创建决策树
def createTree(data,labels):
    classList=list(data['Survived'].values)
    if classList.count(classList[0])==len(classList):
        return classList[0]
    if len(data.columns)==1:
        return majorityClass(classList)
    bestFeature=chooseBestFeatureToSplit(data)
    myTree={bestFeature:{}}
    featVals=data[bestFeature]
    uniqueVals=set(featVals)
    for valus in uniqueVals:
        subLabels=labels[:]
        #print(subLabels)
        new_splitData=splitData(data,bestFeature,valus)
        new_splitData=new_splitData.reset_index()
        new_splitData=new_splitData.drop(['index'],axis=1)
        #print(new_splitData)
        myTree[bestFeature][valus]=createTree(new_splitData,labels)
    return myTree

myTree=createTree(dataSet,dataSet.columns)
print(myTree)

{'Sex': {'male': {'Age2': {0.0: {'SibSp': {0: 1, 1: 1, 2: 1, 3: {'Parch': {1: 0, 2: {'Fare2': {2.0: 0, 3.0: {'Pclass': {3: {'Embarked': {'S': 0}}}}}}}}}}, 1.0: {'Fare2': {0.0: {'Pclass': {1: 0, 2: 0, 3: {'Embarked': {'S': {'SibSp': {0: {'Parch': {0: 0}}, 1: {'Parch': {0: 0, 1: 0}}}}, 'C': {'SibSp': {0: {'Parch': {0: 0}}, 1: 0}}, 'Q': {'SibSp': {0: {'Parch': {0: 0}}, 1: 0}}}}}}, 1.0: {'SibSp': {0: {'Parch': {0: {'Embarked': {'S': {'Pclass': {2: 0, 3: 0}}, 'C': {'Pclass': {2: 0, 3: 0}}, 'Q': 0}}, 1: {'Pclass': {3: {'Embarked': {'S': 0}}}}}}, 1: {'Pclass': {2: 0, 3: {'Parch': {0: 1, 1: 0}}}}, 2: 0}}, 2.0: 0}}, 2.0: {'Pclass': {1: {'SibSp': {0: {'Fare2': {2.0: {'Parch': {0: {'Embarked': {'S': 1, 'C': 0}}, 1: 0}}, 3.0: {'Embarked': {'S': {'Parch': {0: 0}}, 'C': {'Parch': {0: 0}}}}}}, 1: {'Embarked': {'S': {'Parch': {0: {'Fare2': {3.0: 0}}, 1: 1}}, 'C': 1}}}}, 2: {'SibSp': {0: 0, 1: {'Embarked': {'S': {'Fare2': {2.0: {'Parch': {0: 0, 1: 0, 2: 0}}, 3.0: {'Parch': {1: 0}}}}, 'C': 0}}}}, 3: {'S

In [33]:
#使用决策树的分类函数
def classify(inputTree,featLabels,testVec):
    global classLabel
    firstStr=list(inputTree.keys())
    firstStr=firstStr[0]
    #print(firstStr)
    secondDict=inputTree[firstStr]
    #print(secondDict)
    featIndex=featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex]==key:
            if type(secondDict[key]).__name__=='dict':
                classLabel=classify(secondDict[key],featLabels,testVec)
            else:
                classLabel=secondDict[key]
    return classLabel



In [34]:
from math import log
from pandas import DataFrame,Series
import pandas as pd
import operator
import numpy as np
import matplotlib.pyplot as plt

dataTest = pd.read_csv('C:/Users/YT/Desktop/test(2).csv')
#print(dataTest.columns)
#print(dataTest.head(5))
PassengerId=dataTest['PassengerId']
#print(PassengerId)
dataTest=dataTest.drop(['PassengerId'],axis=1)
#print(dataTest.head(5))
s = dataTest['Fare']
for i in range(len(dataTest)):
    if s[i]<7.910400:
        dataTest.ix[i, 'Fare2']=0
    elif s[i]<14.454200:
        dataTest.ix[i, 'Fare2']=1
    elif s[i]<31.000000:
        dataTest.ix[i, 'Fare2']=2
    else:
        dataTest.ix[i, 'Fare2']=3
    if dataTest.ix[i, 'Parch']>2:
        dataTest.ix[i, 'Parch'] = 2
    if dataTest.ix[i, 'SibSp']>3:
        dataTest.ix[i, 'SibSp'] = 3

        
dataTest=dataTest.fillna(dataTest['Age'].median())
#print(dataTest['Age'])
k=dataTest['Age']
for i in range(len(dataTest)):
    if k[i]=='NaN':
        dataSet=dataTest.dropna(axis=0,subset=['Age'])
    elif k[i]<5.000000:
        dataTest.ix[i,'Age2']=0
    elif s[i]<15.000000:
        dataTest.ix[i,'Age2']=1
    elif s[i]<60.000000:
        dataTest.ix[i,'Age2']=2
    else:
        dataTest.ix[i,'Age2']=3
        
#print(dataTest['Age2'])        
        

#删除无用变量
dataTest=dataTest.drop(['Age'],axis=1)
dataTest=dataTest.drop(['Cabin'],axis=1)
dataTest=dataTest.drop(['Name'],axis=1)
dataTest=dataTest.drop(['Ticket'],axis=1)
dataTest=dataTest.drop(['Fare'],axis=1)
print(dataTest.head(3))

dataTest=dataTest.fillna({'Embarked':'S'})
#dataTest=np.array(dataTest)
#print(dataTest)

C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc fo

   Pclass     Sex  SibSp  Parch Embarked  Fare2  Age2
0       3    male      0      0        Q    0.0   1.0
1       3  female      1      0        S    0.0   1.0
2       2    male      0      0        Q    1.0   1.0


In [35]:
myTree=createTree(dataSet,dataSet.columns)
featLables=dataSet.columns.tolist()

for i in range(len(dataSet)):
    a_test=dataSet.ix[i]
    r_p=classify(myTree,featLables,a_test)
    print(r_p)
   

C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


0
1
0
1
0
0
0
0
1
1
1
1
0
0
0
1
0
0
1
1
0
0
1
0
0
0
0
0
1
0
0
1
1
0
0
0
0
0
0
1
0
1
0
1
1
0
0
1
0
1
0
0
1
1
0
0
1
0
1
0
0
1
0
0
0
1
1
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
1
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
1
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
0
0
1
1
0
1
1
0
0
1
0
0
0
0
0
0
0
0
0
0
1
1
0
1
1
0
0
1
0
1
1
0
1
0
0
1
1
0
0
0
0
0
0
0
1
1
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
1
0
0
0
1
0
0
1
1
1
1
1
0
0
0
0
1
0
0
0
1
1
0
0
1
0
1
1
0
0
0
1
0
0
0
0
1
0
0
0
0
1
1
1
0
0
0
0
0
0
1
1
1
1
0
1
0
1
1
1
0
1
1
1
1
0
0
0
1
0
1
1
0
0
1
1
0
1
0
1
1
1
1
1
0
0
1
0
0
1
0
0
1
1
0
0
0
1
1
1
1
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
1
0
0
0
0
1
1
0
1
0
0
0
1
0
1
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
0
0
1
0
1
1
0
1
0
0
1
0
1
1
1
1
1
0
0
0
0
0
0
1
1
1
0
1
0
0
0
0
0
0
1
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
1
0
1
0


In [36]:
import csv

csvfile=open('C:/Users/YT/Desktop/csv.csv','w',newline='')
writer=csv.writer(csvfile)
writer.writerow(['PassengerId','Survived'])
result=[]
#myTree=createTree(dataTest,dataTest.columns)
featLables=dataTest.columns.tolist()

for i in range(len(dataTest)):
    Id=PassengerId[i]
    a_test=dataTest.ix[i]
    r_p=classify(myTree,featLables,a_test)
    print(r_p)
    current_result=(Id,r_p)
    result.append(current_result)
#print(result)
writer.writerows(result)
csvfile.close()


C:\Users\YT\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


0
1
0
0
0
0
1
0
1
0
0
1
1
0
1
1
0
0
0
1
1
1
1
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
1
1
1
0
1
1
0
0
0
1
0
0
0
1
1
0
0
0
0
0
1
0
0
0
1
1
1
1
0
0
1
1
0
0
0
1
0
0
1
0
1
1
0
1
0
0
0
1
0
1
1
0
0
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
1
0
1
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
1
0
1
0
0
1
0
0
1
0
1
1
1
1
1
0
0
1
0
0
0
0
0
0
1
0
0
1
1
1
1
1
0
1
1
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
1
1
0
1
0
1
0
1
0
1
0
0
1
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
1
1
0
1
0
0
0
1
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
1
0
1
1
0
0
0
1
1
1
0
0
1
0
1
1
0
1
1
0
0
1
0
0
1
0
0
1
1
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
1
0
0
1
0
1
0
0
1
0
1
0
1
0
0
1
1
1
1
1
0
0
1
0
0
1
